In [1]:
import os
import sys
import matplotlib.pyplot as plt

os.chdir('..')

import numpy as np
import torch
import pandas as pd

from models.TrainPlanningOperator3D import PlanningOperator3D

from planners import generaterandompos
from planners import AStarPlanner, FMMPlanner, PlanningOperatorPlanner, testplanneronmaps



Load Gibson Environment

In [2]:
Ntotal = 80

ntest = 16
ntrain = 8

sub = 1
Sx = 80
Sy = 80
Sz = 30

maps = np.load('dataset/occupancymaps.npy')
maps = maps[:Ntotal][:ntrain, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]
# maps = maps[0:8,:,:,:]
# maps = np.repeat(maps, repeats=5, axis=0)



Generate Random Positions

In [8]:

startpositions = generaterandompos(maps)
goalpositions = generaterandompos(maps)

Load Model

In [4]:


# Model
modes = 5
width = 24
nlayers = 1

model = PlanningOperator3D(modes, modes, modes, width, nlayers)
model.load_state_dict(torch.load("/mountvol/igib-hexpo-dataset-80-10g/planningoperator_env80_m8_w32_l1_b5_lr3e-3_10g_22sep/n640_lr3.000000e-03_gamma6.000000e-01_wd3.000000e-06_seed5/model3d.ckpt"))
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


# mapx = maps[0,:,:,:]
# startx = startpositions[0,:]
# goalx = goalpositions[0,:]


# success,pathcost,planningtime = AStarPlanner(startx, goalx, mapx)

# print("\nAStar\nSuccess:", success,
#       "\nPathCost:", pathcost,
#       "\nPlanningTime:",planningtime)


# success,pathcost,planningtime = FMMPlanner(startx, goalx, mapx)
# print("\nFMM\nSuccess:", success,
#       "\nPathCost:", pathcost,
#       "\nPlanningTime:",planningtime)


# success,pathcost,tot_time= PlanningOperatorPlanner(startx, goalx, mapx, model)
# print("\nPlanning Operator\nSuccess:", success,
#       "\nPathCost:", pathcost,
#       "\nTotal Time", tot_time)


/tmp/ipykernel_31673/2568948668.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/mountvol/igib-hexpo-dataset-80-10g/planningoperator_en

PlanningOperator3D(
  (fc0): Linear(in_features=3, out_features=24, bias=True)
  (conv0): SpectralConv3d()
  (w0): Conv3d(24, 24, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (fc1): DeepNormMetric(
    (Us): ModuleList(
      (0-1): 2 x Linear(in_features=24, out_features=128, bias=False)
    )
    (Ws): ModuleList(
      (0): ConstrainedLinear(in_features=128, out_features=128, bias=False)
    )
    (activation): MaxReLUPairwiseActivation(
      (avg_pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (output_activation): ConcaveActivation()
    (reduce_metric): ReduceMetric()
  )
)

In [9]:
testplanneronmaps(startpositions,goalpositions,maps,AStarPlanner,printvalues=True)

Average Path Cost: 14.140696070969636 
Average Planning Time: 0.022078901529312134 
Average Success Rate: 1.0


(14.140696070969636, 0.022078901529312134, 1.0)

In [10]:
testplanneronmaps(startpositions,goalpositions,maps,FMMPlanner,printvalues=True)

Average Path Cost: 14.463403282396625 
Average Planning Time: 0.02383008599281311 
Average Success Rate: 1.0


(14.463403282396625, 0.02383008599281311, 1.0)

In [11]:
testplanneronmaps(startpositions,goalpositions,maps,PlanningOperatorPlanner,printvalues=True, model=model)

Average Path Cost: 15.328912592904686 
Average Planning Time: 0.18997585773468018 
Average Success Rate: 1.0


(15.328912592904686, 0.18997585773468018, 1.0)